In [29]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import numpy as np

import os

In [30]:
def parse_trends_timestamp(timestamp):
    datetime_str = timestamp.split(' ')
    return datetime.datetime.fromisoformat(datetime_str[0].split('/')[2]+'-'+datetime_str[0].split('/')[0]+'-'+datetime_str[0].split('/')[1]+ ' ' + datetime_str[1])

In [31]:
def parse_feedback_timestamp(timestamp):
    date_str = timestamp.split('T')[0]
    time_str = timestamp.split('T')[1]

    return datetime.datetime.fromisoformat(date_str + ' ' + time_str[:5])

### Loading Feedback data


In [32]:
feedback_df = pd.read_csv('../data/abschluss-daten/feedback.csv')
device_df = pd.read_csv('../data/abschluss-daten/device.csv')
participant_df = pd.read_csv('../data/abschluss-daten/participant.csv')

In [33]:
for column_name in feedback_df.columns:
    if 'timestamp' in column_name or 'At' in column_name:
        feedback_df[column_name] = feedback_df[column_name].dropna().apply(parse_feedback_timestamp)

for column_name in device_df.columns:
    if 'timestamp' in column_name or 'At' in column_name:
        device_df[column_name] = device_df[column_name].dropna().apply(parse_feedback_timestamp)

for column_name in participant_df.columns:
    if 'timestamp' in column_name or 'At' in column_name:
        participant_df[column_name] = participant_df[column_name].dropna().apply(parse_feedback_timestamp)


### Loading Temperature & Air quality data

Loading data is done as a batch, appending each file read to a list

In [34]:
#0EG
dir_list = os.listdir('../data/Trends Smart City/0EG')
_0_eg_temp_dfs = [] #Store
_0_eg_qual_dfs = []
for filename in dir_list:
    if 'Lufttemp' in filename:
        _0_eg_temp_dfs.append(pd.read_csv('../data/Trends Smart City/0EG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','temperature','reduced']))
        _0_eg_temp_dfs[-1]['timestamp'] = _0_eg_temp_dfs[-1]['timestamp'].apply(parse_trends_timestamp)
    else:
        _0_eg_qual_dfs.append(pd.read_csv('../data/Trends Smart City/0EG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','quality','reduced']))
        _0_eg_qual_dfs[-1]['timestamp'] = _0_eg_qual_dfs[-1]['timestamp'].apply(parse_trends_timestamp)

#1OG
dir_list = os.listdir('../data/Trends Smart City/1OG')
_1_og_temp_dfs = []
_1_og_qual_dfs = []
for filename in dir_list:
    if 'Lufttemp' in filename:
        _1_og_temp_dfs.append(pd.read_csv('../data/Trends Smart City/1OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','temperature','reduced']))
        _1_og_temp_dfs[-1]['timestamp'] = _1_og_temp_dfs[-1]['timestamp'].apply(parse_trends_timestamp)
    else:
        _1_og_qual_dfs.append(pd.read_csv('../data/Trends Smart City/1OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','quality','reduced']))
        _1_og_qual_dfs[-1]['timestamp'] = _1_og_qual_dfs[-1]['timestamp'].apply(parse_trends_timestamp)



#2OG
dir_list = os.listdir('../data/Trends Smart City/2OG')
_2_og_temp_dfs = []
_2_og_qual_dfs = []
for filename in dir_list:
    if 'Lufttemp' in filename:
        _2_og_temp_dfs.append(pd.read_csv('../data/Trends Smart City/2OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','temperature','reduced']))
        _2_og_temp_dfs[-1]['timestamp'] = _2_og_temp_dfs[-1]['timestamp'].apply(parse_trends_timestamp)
    else:
        _2_og_qual_dfs.append(pd.read_csv('../data/Trends Smart City/2OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','quality','reduced']))
        _2_og_qual_dfs[-1]['timestamp'] = _2_og_qual_dfs[-1]['timestamp'].apply(parse_trends_timestamp)


#3OG
dir_list = os.listdir('../data/Trends Smart City/3OG')
_3_og_temp_dfs = []
_3_og_qual_dfs = []
for filename in dir_list:
    if 'Lufttemp' in filename:
        _3_og_temp_dfs.append(pd.read_csv('../data/Trends Smart City/3OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','temperature','reduced']))
        _3_og_temp_dfs[-1]['timestamp'] = _3_og_temp_dfs[-1]['timestamp'].apply(parse_trends_timestamp)
    else:
        _3_og_qual_dfs.append(pd.read_csv('../data/Trends Smart City/3OG/'+filename,sep=';',header=0,usecols = [i for i in range(3)],names=['timestamp','quality','reduced']))
        _3_og_qual_dfs[-1]['timestamp'] = _3_og_qual_dfs[-1]['timestamp'].apply(parse_trends_timestamp)



Merging participant and device datasets to link each participant to a room 

In [39]:
participant_df['participant_id'] = participant_df['_id']
participant_df = participant_df.drop(columns=['_id'])

device_df['device_id'] = device_df['_id']
device_df = device_df.drop(columns=['_id'])


In [41]:
participant_device_df = pd.merge(device_df,participant_df,how='left',left_on='device_id',right_on='gdpr_device').drop(['gdpr_device'], axis=1)

In [49]:
participant_device_df = participant_device_df[['device_id','participant_id', 'roomName']].dropna()

Merging feedback and room based on participant

In [52]:
feedback_room_df = pd.merge(feedback_df,participant_device_df,how='left',left_on='participant',right_on='participant_id').drop(['participant_id'], axis=1)

In [55]:
feedback_room_df[['hourly_airQuality_timestamp', 'hourly_airQuality_value','device_id','participant', 'roomName']].dropna()

,hourly_airQuality_timestamp,hourly_airQuality_value,device_id,participant,roomName
0,2022-09-07 18:11:00,1.0,6318de412c5704a9e531d59d,6318dea612223a195314b5ee,Stefan
1,2022-09-08 11:17:00,1.0,63186f95a505031fc7fc2550,6318dea512223a195314b5eb,VS 1.06
4,2022-09-08 12:08:00,3.0,63186baaa505031fc7fc253d,6318dea612223a195314b603,VS 0.24
5,2022-09-09 07:14:00,5.0,63186f95a505031fc7fc2550,6318dea512223a195314b5eb,VS 1.06
6,2022-09-14 06:58:00,1.0,63186922a505031fc7fc2532,6318dea612223a195314b5f0,VS 2.04
...,...,...,...,...,...
746,2022-10-24 12:05:00,3.0,63186922a505031fc7fc2532,6318dea712223a195314b60c,VS 2.04
747,2022-10-25 06:29:00,2.0,63186922a505031fc7fc2532,6318dea712223a195314b60c,VS 2.04
748,2022-10-25 11:10:00,2.0,63186922a505031fc7fc2532,6318dea712223a195314b60c,VS 2.04
749,2022-10-25 11:10:00,2.0,63186922a505031fc7fc2532,6318dea612223a195314b5f0,VS 2.04


### Splitting feedback data

In [ ]:
daily_airDraft_df = feedback_df[['daily_airDraft_timestamp','daily_airDraft_value']].dropna()
daily_ambientSounds_df = feedback_df[['daily_ambientSounds_timestamp','daily_ambientSounds_value']].dropna()
daily_artificialLight_df = feedback_df[['daily_artificialLight_timestamp','daily_artificialLight_value']].dropna()
daily_dayLight_df = feedback_df[['daily_dayLight_timestamp','daily_dayLight_value']].dropna()
daily_smell_df = feedback_df[['daily_smell_timestamp','daily_smell_value']].dropna()
daily_studentsAdherence_df = feedback_df[['daily_studentsAdherence_timestamp', 'daily_studentsAdherence_value']].dropna()
daily_studentsFocused_df = feedback_df[['daily_studentsFocused_timestamp', 'daily_studentsFocused_value','participant']].dropna()
daily_studentsIrritated_df = feedback_df[['daily_studentsIrritated_timestamp', 'daily_studentsIrritated_value']].dropna()
daily_studentsMotivated_df =feedback_df[['daily_studentsMotivated_timestamp', 'daily_studentsMotivated_value']].dropna()

hourly_airQuality_df = feedback_df[['hourly_airQuality_timestamp', 'hourly_airQuality_value']].dropna()
hourly_humidity_df = feedback_df[['hourly_humidity_timestamp', 'hourly_humidity_value']].dropna()
hourly_indoorClimate_df = feedback_df[['hourly_indoorClimate_timestamp', 'hourly_indoorClimate_value']].dropna()
hourly_temperature_df = feedback_df[['hourly_temperature_timestamp', 'hourly_temperature_value','participant']].dropna()

In [ ]:

# daily_studentsFocused_df['timestamp'] = daily_studentsFocused_df['daily_studentsFocused_timestamp']

# #Setting a joint index of "timestamp" and "participant"
# daily_studentsFocused_df = daily_studentsFocused_df.set_index(['timestamp','participant'])

# daily_studentsFocused_df = daily_studentsFocused_df.drop(columns=['daily_studentsFocused_timestamp'])

In [ ]:
# device_df = device_df.set_index(['_id','createdAt', 'updatedAt'])
# participant_df = participant_df.set_index(['_id','createdAt', 'updatedAt'])

# #Probabbly I have to add 'createdAt', 'updatedAt' to feedback index as well

In [ ]:
# device_participant_df = device_df.join(participant_df,how='outer')